# FeatureBook Alternative 1

In [1]:
import gc
import numpy as np
import pandas as pd
import xgboost as xgb

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

## MultipleNN Angle Features

In [2]:
def generate_angles(df, df_structures):
    # Map XYZ-coordinates to the dataframe
    def map_atom_info(df, df_merge, atom_idx):
        df = pd.merge(df, df_merge.drop_duplicates(subset=['molecule_name', 'atom_index']), how = 'left',
                      left_on  = ['molecule_name', f'atom_index_{atom_idx}'],
                      right_on = ['molecule_name',  'atom_index'])

        df = df.drop('atom_index', axis=1)

        return df

    for atom_idx in [0, 1]:
        df = map_atom_info(df, df_structures, atom_idx)
        df = df.rename(columns={
            'atom': f'atom_{atom_idx}',
            'x': f'x_{atom_idx}',
            'y': f'y_{atom_idx}',
            'z': f'z_{atom_idx}'})
        
        df_structures['c_x'] = df_structures.groupby('molecule_name')['x'].transform('mean')
        df_structures['c_y'] = df_structures.groupby('molecule_name')['y'].transform('mean')
        df_structures['c_z'] = df_structures.groupby('molecule_name')['z'].transform('mean')
        df_structures['atom_n'] = df_structures.groupby('molecule_name')['atom_index'].transform('max')
    
    # Calculate initial distances
    def calculate_init_distances(df):
        df['dx'] = df['x_1'] - df['x_0']
        df['dy'] = df['y_1'] - df['y_0']
        df['dz'] = df['z_1'] - df['z_0']
        df['distance'] = (df['dx']**2 + df['dy']**2 + df['dz']**2)**(1/2)
        return df

    df = calculate_init_distances(df)
    
    # Extend the distance calculations
    def extended_distance_calculations(df):
        df_temp = df.loc[:, ["molecule_name","atom_index_0","atom_index_1","distance","x_0","y_0","z_0","x_1","y_1","z_1"]].copy()
        df_temp_ = df_temp.copy()
        df_temp_ = df_temp_.rename(columns={
            'atom_index_0': 'atom_index_1',
            'atom_index_1': 'atom_index_0',
            'x_0': 'x_1', 'y_0': 'y_1',
            'z_0': 'z_1', 'x_1': 'x_0',
            'y_1': 'y_0', 'z_1': 'z_0'})
        
        df_temp_all = pd.concat((df_temp, df_temp_), axis=0)
        
        df_temp_all["min_distance"] = df_temp_all.groupby(['molecule_name', 'atom_index_0'])['distance'].transform('min')
        df_temp_all["max_distance"] = df_temp_all.groupby(['molecule_name', 'atom_index_0'])['distance'].transform('max')
        
        df_temp = df_temp_all[df_temp_all["min_distance"] == df_temp_all["distance"]].copy()
        df_temp = df_temp.drop(['x_0', 'y_0', 'z_0', 'min_distance'], axis=1)
        df_temp = df_temp.rename(columns={
            'atom_index_0': 'atom_index',
            'atom_index_1': 'atom_index_closest',
            'distance': 'distance_closest',
            'x_1': 'x_closest',
            'y_1': 'y_closest',
            'z_1': 'z_closest'})
        
        for atom_idx in [0, 1]:
            df = map_atom_info(df, df_temp, atom_idx)
            df = df.rename(columns={
                'atom_index_closest': f'atom_index_closest_{atom_idx}',
                'distance_closest': f'distance_closest_{atom_idx}',
                'x_closest': f'x_closest_{atom_idx}',
                'y_closest': f'y_closest_{atom_idx}',
                'z_closest': f'z_closest_{atom_idx}'})
        
        df_temp = df_temp_all[df_temp_all["max_distance"] == df_temp_all["distance"]].copy()
        df_temp = df_temp.drop(['x_0', 'y_0', 'z_0', 'max_distance'], axis=1)
        df_temp = df_temp.rename(columns={
            'atom_index_0': 'atom_index',
            'atom_index_1': 'atom_index_farthest',
            'distance': 'distance_farthest',
            'x_1': 'x_farthest',
            'y_1': 'y_farthest',
            'z_1': 'z_farthest'})
        
        for atom_idx in [0, 1]:
            df = map_atom_info(df, df_temp, atom_idx)
            df = df.rename(columns={
                'atom_index_farthest': f'atom_index_farthest_{atom_idx}',
                'distance_farthest': f'distance_farthest_{atom_idx}',
                'x_farthest': f'x_farthest_{atom_idx}',
                'y_farthest': f'y_farthest_{atom_idx}',
                'z_farthest': f'z_farthest_{atom_idx}'})
        
        return df
    
    df = extended_distance_calculations(df)
    
    # Add angle features
    def add_angles(df):
        df["distance_center0"]=((df['x_0']-df['c_x'])**2+(df['y_0']-df['c_y'])**2+(df['z_0']-df['c_z'])**2)**(1/2)
        df["distance_center1"]=((df['x_1']-df['c_x'])**2+(df['y_1']-df['c_y'])**2+(df['z_1']-df['c_z'])**2)**(1/2)
        df["distance_c0"]=((df['x_0']-df['x_closest_0'])**2+(df['y_0']-df['y_closest_0'])**2+(df['z_0']-df['z_closest_0'])**2)**(1/2)
        df["distance_c1"]=((df['x_1']-df['x_closest_1'])**2+(df['y_1']-df['y_closest_1'])**2+(df['z_1']-df['z_closest_1'])**2)**(1/2)
        df["distance_f0"]=((df['x_0']-df['x_farthest_0'])**2+(df['y_0']-df['y_farthest_0'])**2+(df['z_0']-df['z_farthest_0'])**2)**(1/2)
        df["distance_f1"]=((df['x_1']-df['x_farthest_1'])**2+(df['y_1']-df['y_farthest_1'])**2+(df['z_1']-df['z_farthest_1'])**2)**(1/2)
        df["vec_center0_x"]=(df['x_0']-df['c_x'])/(df["distance_center0"]+1e-10)
        df["vec_center0_y"]=(df['y_0']-df['c_y'])/(df["distance_center0"]+1e-10)
        df["vec_center0_z"]=(df['z_0']-df['c_z'])/(df["distance_center0"]+1e-10)
        df["vec_center1_x"]=(df['x_1']-df['c_x'])/(df["distance_center1"]+1e-10)
        df["vec_center1_y"]=(df['y_1']-df['c_y'])/(df["distance_center1"]+1e-10)
        df["vec_center1_z"]=(df['z_1']-df['c_z'])/(df["distance_center1"]+1e-10)
        df["vec_c0_x"]=(df['x_0']-df['x_closest_0'])/(df["distance_c0"]+1e-10)
        df["vec_c0_y"]=(df['y_0']-df['y_closest_0'])/(df["distance_c0"]+1e-10)
        df["vec_c0_z"]=(df['z_0']-df['z_closest_0'])/(df["distance_c0"]+1e-10)
        df["vec_c1_x"]=(df['x_1']-df['x_closest_1'])/(df["distance_c1"]+1e-10)
        df["vec_c1_y"]=(df['y_1']-df['y_closest_1'])/(df["distance_c1"]+1e-10)
        df["vec_c1_z"]=(df['z_1']-df['z_closest_1'])/(df["distance_c1"]+1e-10)
        df["vec_f0_x"]=(df['x_0']-df['x_farthest_0'])/(df["distance_f0"]+1e-10)
        df["vec_f0_y"]=(df['y_0']-df['y_farthest_0'])/(df["distance_f0"]+1e-10)
        df["vec_f0_z"]=(df['z_0']-df['z_farthest_0'])/(df["distance_f0"]+1e-10)
        df["vec_f1_x"]=(df['x_1']-df['x_farthest_1'])/(df["distance_f1"]+1e-10)
        df["vec_f1_y"]=(df['y_1']-df['y_farthest_1'])/(df["distance_f1"]+1e-10)
        df["vec_f1_z"]=(df['z_1']-df['z_farthest_1'])/(df["distance_f1"]+1e-10)
        df["vec_x"]=(df['x_1']-df['x_0'])/df["distance"]
        df["vec_y"]=(df['y_1']-df['y_0'])/df["distance"]
        df["vec_z"]=(df['z_1']-df['z_0'])/df["distance"]
        df["cos_c0_c1"]=df["vec_c0_x"]*df["vec_c1_x"]+df["vec_c0_y"]*df["vec_c1_y"]+df["vec_c0_z"]*df["vec_c1_z"]
        df["cos_f0_f1"]=df["vec_f0_x"]*df["vec_f1_x"]+df["vec_f0_y"]*df["vec_f1_y"]+df["vec_f0_z"]*df["vec_f1_z"]
        df["cos_center0_center1"]=df["vec_center0_x"]*df["vec_center1_x"]+df["vec_center0_y"]*df["vec_center1_y"]+df["vec_center0_z"]*df["vec_center1_z"]
        df["cos_c0"]=df["vec_c0_x"]*df["vec_x"]+df["vec_c0_y"]*df["vec_y"]+df["vec_c0_z"]*df["vec_z"]
        df["cos_c1"]=df["vec_c1_x"]*df["vec_x"]+df["vec_c1_y"]*df["vec_y"]+df["vec_c1_z"]*df["vec_z"]
        df["cos_f0"]=df["vec_f0_x"]*df["vec_x"]+df["vec_f0_y"]*df["vec_y"]+df["vec_f0_z"]*df["vec_z"]
        df["cos_f1"]=df["vec_f1_x"]*df["vec_x"]+df["vec_f1_y"]*df["vec_y"]+df["vec_f1_z"]*df["vec_z"]
        df["cos_center0"]=df["vec_center0_x"]*df["vec_x"]+df["vec_center0_y"]*df["vec_y"]+df["vec_center0_z"]*df["vec_z"]
        df["cos_center1"]=df["vec_center1_x"]*df["vec_x"]+df["vec_center1_y"]*df["vec_y"]+df["vec_center1_z"]*df["vec_z"]
        # original
        df=df.drop(['vec_c0_x','vec_c0_y','vec_c0_z','vec_c1_x','vec_c1_y','vec_c1_z',
                    'vec_f0_x','vec_f0_y','vec_f0_z','vec_f1_x','vec_f1_y','vec_f1_z',
                    'vec_center0_x','vec_center0_y','vec_center0_z','vec_center1_x','vec_center1_y','vec_center1_z',
                    'vec_x','vec_y','vec_z'], axis=1)
        # extra 
        df=df.drop(['dy', 'dx', 'dz',
                    'x_0', 'x_1', 'c_x', 'x_closest_0', 'x_closest_1', 'x_farthest_0', 'x_farthest_1',
                    'y_0', 'y_1', 'c_y', 'y_closest_0', 'y_closest_1', 'y_farthest_0', 'y_farthest_1',
                    'z_0', 'z_1', 'c_z', 'z_closest_0', 'z_closest_1', 'z_farthest_0', 'z_farthest_1'], axis=1)
        return df
    
    df = add_angles(df)
    
    return df

## General Methods

In [3]:
coupling_types = ["1JHC", "1JHN", "2JHH", "2JHC", "2JHN", "3JHH", "3JHC", "3JHN"]

def select_wanted(df, wanted_rows):
    df = df[df["type"] == wanted_rows]
    return df

def memory_optimization(dfs):
    for df in dfs:
        del df
    gc.collect()

def merge_multiple(df, df_merge, atom_idx):
    df = pd.merge(df, df_merge, how = 'left',
                  left_on  = ['molecule_name', f'atom_index_{atom_idx}'],
                  right_on = ['molecule_name',  'atom_index'])
    
    df = df.drop('atom_index', axis=1)
    return df

def merge_single(df, df_merge):
    df = pd.merge(df, df_merge, how = 'left',
                  left_on  = ['molecule_name'],
                  right_on = ['molecule_name'])
        
    return df

def merge_custom(df, df_merge):
    df = pd.merge(df, df_merge, how = 'left',
                 left_on  = ['id', 'molecule_name'],
                 right_on = ['id', 'molecule_name'])
    
    return df

def drop_col(df, cols):
    columns = list(df.columns.values)
    
    for col in cols:
        if col in columns:
            df = df.drop(col, axis=1)
            
    return df

## Append Custom Features to DataFrame

In [4]:
def loader(df, train_or_test):
    # Sorted distances
    df_sorted_distances = pd.read_csv('./input/generated/sorted_distances.csv')
    df = merge_multiple(df, df_sorted_distances, 0)
    df = merge_multiple(df, df_sorted_distances, 1)
    memory_optimization([df_sorted_distances])
    
    # Dipole moments
    df_dipole_moments = pd.read_csv(f'./input/generated/{train_or_test}_ob_dipoles_mmff44.csv')
    df_dipole_moments = df_dipole_moments.loc[:, ~df_dipole_moments.columns.str.contains('^Unnamed')]
    df_dipole_moments['size'] = (df_dipole_moments['X']**2 + df_dipole_moments['Y']**2 + df_dipole_moments['Z']**2)**(1/2)
    df = merge_single(df, df_dipole_moments)
    memory_optimization([df_dipole_moments])

    # Mulliken charges
    df_mulliken_charges = pd.read_csv(f'./input/generated/best_ob_mulliken_{train_or_test}.csv')
    df_mulliken_charges = df_mulliken_charges.loc[:, ~df_mulliken_charges.columns.str.contains('^Unnamed')]
    df = merge_multiple(df, df_mulliken_charges, 0)
    df = merge_multiple(df, df_mulliken_charges, 1)
    memory_optimization([df_mulliken_charges])
    
    # Number of atoms and max distances
    df_natoms_maxdist = pd.read_csv(f'./input/generated/natoms_maxdist.csv')
    df = merge_multiple(df, df_natoms_maxdist, 0)
    df = df.drop(['atom'], axis = 1)
    memory_optimization([df_natoms_maxdist])

    # Coupling distances
    df_coupling_distances = pd.read_csv(f'./input/generated/coupling_distances.csv')
    df = merge_custom(df, df_coupling_distances)
    memory_optimization([df_coupling_distances])
    
    df = drop_col(df, ['atom_index_0', 'atom_index_1', 'atom_index_closest_0', 'atom_index_closest_1', 
                       'atom_index_farthest_0', 'atom_index_farthest_1', 'id', 'X', 'Y', 'Z'])
    
    return df

## Convert Data Types

In [5]:
def convert_df(df):
    atom_representation = {
        'H': 1, 'C': 6, 'N': 7, 'O': 8, 'F': 9,
    }

    coupling_representation = {
        '1JHC': 1, '1JHN': 2, '2JHH': 3, '2JHC': 4, '2JHN': 5, '3JHH': 6, '3JHC': 7, '3JHN': 8,
    }

    df = df.replace({
        'type': coupling_representation,
        'atom_x': atom_representation,
        'atom_y': atom_representation,
        'atom_0': atom_representation,
        'atom_1': atom_representation,
    })
    df.type = df.type.astype('int64')
    df.atom_x = df.atom_x.astype('int64')
    df.atom_y = df.atom_y.astype('int64')
    df.atom_0 = df.atom_0.astype('int64')
    df.atom_1 = df.atom_1.astype('int64')
    
    df['molecule_name'] = df['molecule_name'].astype('category')
    df_cats = df.select_dtypes(['category']).columns
    df[df_cats] = df[df_cats].apply(lambda x: x.cat.codes)
    
    return df

## Column Drop Test

In [6]:
def col_dropping(df):
    df = df.drop(['molecule_name'], axis=1)
    df = df.drop(['atom_n'], axis=1)
    
    # Drop distances
    df = drop_col(df, ['d10_x', 'd11_x', 'd12_x', 'd13_x', 'd14_x', 'd15_x', 'd16_x', 'd17_x', 'd18_x', 'd19_x', 'd20_x', 'd21_x', 'd22_x', 'd23_x', 'd24_x', 'd25_x', 'd26_x', 'd27_x', 'd28_x',
                  'd10_y', 'd11_y', 'd12_y', 'd13_y', 'd14_y', 'd15_y', 'd16_y', 'd17_y', 'd18_y', 'd19_y', 'd20_y', 'd21_y', 'd22_y', 'd23_y', 'd24_y', 'd25_y', 'd26_y', 'd27_y', 'd28_y'])
    
    # Drop types
    df = drop_col(df, ['t10_x', 't11_x', 't12_x', 't13_x', 't14_x', 't15_x', 't16_x', 't17_x', 't18_x', 't19_x', 't20_x', 't21_x', 't22_x', 't23_x', 't24_x', 't25_x', 't26_x', 't27_x', 't28_x',
                  't10_y', 't11_y', 't12_y', 't13_y', 't14_y', 't15_y', 't16_y', 't17_y', 't18_y', 't19_y', 't20_y', 't21_y', 't22_y', 't23_y', 't24_y', 't25_y', 't26_y', 't27_y', 't28_y'])
    
    df = drop_col(df, ['type', 'atom_0', 'atom_1', 'eem_x', 'eem_y', 'X', 'Y', 'Z'])
    
    return df

## Features Master Organizer

In [7]:
def feature_master(coupling_type):
    # Load structures
    df_structures = pd.read_csv('./input/structures.csv')
    
    # Generate train for current coupling type
    df_train = pd.read_csv('./input/train.csv')
    df_train = select_wanted(df_train, coupling_type)
    df_train = generate_angles(df_train, df_structures)
    df_train = loader(df_train, 'train')
    df_train = convert_df(df_train)
    df_train = col_dropping(df_train)
    print(df_train.head(25))
    
    df_structures = None
    df_structures = pd.read_csv('./input/structures.csv')
    
    # Generate test for current coupling type
    df_test = pd.read_csv('./input/test.csv')
    df_test = select_wanted(df_test, coupling_type)
    df_test = generate_angles(df_test, df_structures)
    df_test = loader(df_test, 'test')
    df_test = convert_df(df_test)
    df_test = col_dropping(df_test)
    print(df_test.head(25))
    
    # Return results
    return df_train, df_test

## Split Training and Validation

In [8]:
def split(df_train):
    train_X, validation_X = train_test_split(df_train, test_size = 0.1, random_state = 0)

    train_X = train_X.reset_index()
    validation_X = validation_X.reset_index()

    train_y = train_X['scalar_coupling_constant']
    train_y = train_y.replace([np.inf, -np.inf], np.nan)
    train_y = train_y.reset_index()
    train_y = train_y.drop(['index'], axis = 1)
    validation_y = validation_X['scalar_coupling_constant']
    validation_y = validation_y.replace([np.inf, -np.inf], np.nan)
    validation_y = validation_y.reset_index()
    validation_y = validation_y.drop(['index'], axis = 1)

    train_X = train_X.drop('scalar_coupling_constant', axis = 1)
    validation_X = validation_X.drop('scalar_coupling_constant', axis = 1)
    
    train_X = train_X.drop(['index'], axis = 1)
    validation_X = validation_X.drop(['index'], axis = 1)

    print(train_X.head())
    print(train_y.head())
    print(validation_X.head())
    print(validation_y.head())
    
    return train_X, train_y, validation_X, validation_y

## XGB Training

In [ ]:
def xgb_train(coupling_type, train_X, train_y, validation_X, validation_y):
    model_name_wrt = ('./featurebook/xgb_30.07/featurebook_%s.hdf5' % coupling_type)

    xgb_model = xgb.XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.5, gamma=0, importance_type='gain',
       learning_rate=0.02, max_delta_step=0, max_depth=9, # was 9
       min_child_weight=5, missing=None, n_estimators=10000, n_jobs=-1,
       nthread=None, objective='reg:squarederror', random_state=101, reg_alpha=0.5,
       reg_lambda=0.2, scale_pos_weight=1, seed=None, silent=False, 
       subsample=1)

    xgb_model.fit(train_X, train_y, eval_set = [(validation_X, validation_y)], eval_metric = 'mae', 
              early_stopping_rounds = 32, verbose = True)   
    xgb_model.save_model(model_name_wrt)
    
    cv_predict = xgb_model.predict(validation_X)
    accuracy = np.mean(np.abs(validation_y.iloc[:, 0]-cv_predict))
    cv_score = np.log(accuracy)
    
    print((validation_y.iloc[:, 0]).shape, cv_predict.shape)
    print(f"cv score for {coupling_type} is {cv_score}\n")
    print(train_X.columns)
    print(xgb_model.feature_importances_)
    
    return xgb_model, model_name_wrt, cv_score

In [ ]:
def importance(xgb_model, train_X):
    input_features = train_X.columns.values

    import operator
    feat_imp = xgb_model.feature_importances_
    np.split(feat_imp, len(input_features))
    feat_imp_dict = {}
    for i in range(0, len(input_features)):
        feat_imp_dict[feat_imp[i]]=input_features[i]

    sorted_feats = sorted(feat_imp_dict.items(), key=operator.itemgetter(0))

    for i in range( len(sorted_feats)-1, 0, -1):
        print(sorted_feats[i])

In [ ]:
def make_prediction(xgb_model, model_name, coupling_type, df_submission, df_test):
    test_predict = xgb_model.predict(df_test)
    test_predict = list(test_predict)
    
    new_results = df_test[['id']].copy()
    new_results.insert(1, "scalar_coupling_constant", test_predict, True)
    print(new_results.head(15))
    
    df_submission.loc[df_submission.id.isin(new_results.id), ['scalar_coupling_constant']] = new_results[['scalar_coupling_constant']].values
    print(df_submission.head(25))
    
    return df_submission

## Master Cell

In [ ]:
csv_file = './submissions/submission_31-07-2019.csv'

best_accuracy_scores = {
    "1JHC": -0.01916, "1JHN": -1.0809040175903528, "2JHH": -1.27420, "2JHC": -0.685838,
    "2JHN": -1.12772, "3JHH": -0.11169, "3JHC": 0.079127, "3JHN": -1.103490,
}

for coupling_type in coupling_types:
    # Get training and test dataframes
    df_train, df_test = feature_master(coupling_type)
    
    # Split into train and validation
    train_X, train_y, validation_X, validation_y = split(df_train)
    memory_optimization([df_train])
    
    # Train
    xgb_model, model_name, cv_score = xgb_train(coupling_type, train_X, train_y, validation_X, validation_y)
    
    # Feature importance
    importance(xgb_model, train_X)
    memory_optimization([train_X, train_y, validation_X, validation_y])
    
    # Make prediction
    if cv_score < best_accuracy_scores[coupling_type]:
        # Load submission file
        df_submission = pd.read_csv(csv_file)
        
        # Make submission
        df_submission_temp = make_prediction(xgb_model, model_name, coupling_type, df_submission, df_test)
        df_submission = df_submission_temp
        
        # Save submission file
        print(f'Saving new results to submission file: {csv_file}')
        df_submission.to_csv(csv_file, index=False)
        df_submission = None

C:\Users\frbak\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:46: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



    scalar_coupling_constant  atom_n  distance  distance_closest_0  \
0                    32.6889       3  1.017190            1.017190   
1                    32.6891       3  1.017187            1.017187   
2                    32.6905       3  1.017208            1.017208   
3                    55.5252       5  1.007511            1.007511   
4                    54.7359       5  1.004933            1.004933   
5                    54.0640       8  1.004771            1.004771   
6                    56.1860       8  1.006952            1.006952   
7                    37.7190       7  1.014969            1.014969   
8                    38.3495       7  1.015277            1.015277   
9                    59.3010       9  1.004448            1.004448   
10                   57.3979       8  1.005982            1.005982   
11                   58.1002       7  1.006813            1.006813   
12                   57.2400       7  1.004746            1.004746   
13                  

    atom_n  distance  distance_closest_0  max_distance_x  distance_closest_1  \
0        7  1.009036            1.009036        1.009036            1.008709   
1        7  1.008709            1.008709        1.008709            1.008709   
2        7  1.009039            1.009039        1.009039            1.008711   
3        7  1.008711            1.008711        1.008711            1.008711   
4        8  1.006898            1.006898        1.006898            1.006898   
5        7  1.007077            1.007077        1.007077            1.005058   
6        7  1.005058            1.005058        1.005058            1.005058   
7       11  1.004926            1.004926        1.004926            1.004926   
8       11  1.006899            1.006899        1.006899            1.004926   
9       11  1.005559            1.005559        1.005559            1.005559   
10       9  1.005768            1.005768        1.005768            1.005248   
11       9  1.005248            1.005248

[0]	validation_0-mae:42.0936
Will train until validation_0-mae hasn't improved in 32 rounds.
[1]	validation_0-mae:37.8777
[2]	validation_0-mae:34.0882
[3]	validation_0-mae:30.6749
[4]	validation_0-mae:27.6041
[5]	validation_0-mae:24.8402
[6]	validation_0-mae:22.3534
[7]	validation_0-mae:20.1164
[8]	validation_0-mae:18.1012
[9]	validation_0-mae:16.289
[10]	validation_0-mae:14.66
[11]	validation_0-mae:13.1934
[12]	validation_0-mae:11.8747
[13]	validation_0-mae:10.6854
[14]	validation_0-mae:9.61572
[15]	validation_0-mae:8.65407
[16]	validation_0-mae:7.78869
[17]	validation_0-mae:7.00977
[18]	validation_0-mae:6.30799
[19]	validation_0-mae:5.67629
[20]	validation_0-mae:5.10708
[21]	validation_0-mae:4.59711
[22]	validation_0-mae:4.13832
[23]	validation_0-mae:3.72548
[24]	validation_0-mae:3.35533
[25]	validation_0-mae:3.02259
[26]	validation_0-mae:2.72363
[27]	validation_0-mae:2.4544
[28]	validation_0-mae:2.21413
[29]	validation_0-mae:2.00023
[30]	validation_0-mae:1.80931
[31]	validation_0-ma

[260]	validation_0-mae:0.372349
[261]	validation_0-mae:0.372324
[262]	validation_0-mae:0.37228
[263]	validation_0-mae:0.372192
[264]	validation_0-mae:0.372015
[265]	validation_0-mae:0.371791
[266]	validation_0-mae:0.371728
[267]	validation_0-mae:0.371653
[268]	validation_0-mae:0.371481
[269]	validation_0-mae:0.371349
[270]	validation_0-mae:0.371195
[271]	validation_0-mae:0.371169
[272]	validation_0-mae:0.371022
[273]	validation_0-mae:0.370926
[274]	validation_0-mae:0.370764
[275]	validation_0-mae:0.370654
[276]	validation_0-mae:0.37057
[277]	validation_0-mae:0.370513
[278]	validation_0-mae:0.370294
[279]	validation_0-mae:0.370157
[280]	validation_0-mae:0.370051
[281]	validation_0-mae:0.369896
[282]	validation_0-mae:0.369641
[283]	validation_0-mae:0.369235
[284]	validation_0-mae:0.369183
[285]	validation_0-mae:0.369111
[286]	validation_0-mae:0.368989
[287]	validation_0-mae:0.368817
[288]	validation_0-mae:0.368681
[289]	validation_0-mae:0.368508
[290]	validation_0-mae:0.368235
[291]	vali

...

[2779]	validation_0-mae:0.179381
[2780]	validation_0-mae:0.179379
[2781]	validation_0-mae:0.179376
[2782]	validation_0-mae:0.179376
[2783]	validation_0-mae:0.179373
[2784]	validation_0-mae:0.179365
[2785]	validation_0-mae:0.179365
[2786]	validation_0-mae:0.179363
[2787]	validation_0-mae:0.179362
[2788]	validation_0-mae:0.179359
[2789]	validation_0-mae:0.179357
[2790]	validation_0-mae:0.179356
[2791]	validation_0-mae:0.179354
[2792]	validation_0-mae:0.179351
[2793]	validation_0-mae:0.17935
[2794]	validation_0-mae:0.17935
[2795]	validation_0-mae:0.179349
[2796]	validation_0-mae:0.179345
[2797]	validation_0-mae:0.179338
[2798]	validation_0-mae:0.179329
[2799]	validation_0-mae:0.179326
[2800]	validation_0-mae:0.179319
[2801]	validation_0-mae:0.179314
[2802]	validation_0-mae:0.179311
[2803]	validation_0-mae:0.179309
[2804]	validation_0-mae:0.179308
[2805]	validation_0-mae:0.1793
[2806]	validation_0-mae:0.179295
[2807]	validation_0-mae:0.17929
[2808]	validation_0-mae:0.179288
[2809]	validati

[3029]	validation_0-mae:0.178486
[3030]	validation_0-mae:0.178482
[3031]	validation_0-mae:0.178479
[3032]	validation_0-mae:0.178476
[3033]	validation_0-mae:0.178475
[3034]	validation_0-mae:0.178475
[3035]	validation_0-mae:0.178469
[3036]	validation_0-mae:0.178467
[3037]	validation_0-mae:0.178463
[3038]	validation_0-mae:0.178456
[3039]	validation_0-mae:0.178456
[3040]	validation_0-mae:0.178453
[3041]	validation_0-mae:0.178448
[3042]	validation_0-mae:0.178445
[3043]	validation_0-mae:0.178443
[3044]	validation_0-mae:0.178443
[3045]	validation_0-mae:0.178438
[3046]	validation_0-mae:0.178435
[3047]	validation_0-mae:0.178432
[3048]	validation_0-mae:0.178427
[3049]	validation_0-mae:0.178424
[3050]	validation_0-mae:0.178416
[3051]	validation_0-mae:0.178408
[3052]	validation_0-mae:0.178405
[3053]	validation_0-mae:0.178403
[3054]	validation_0-mae:0.178399
[3055]	validation_0-mae:0.178394
[3056]	validation_0-mae:0.17839
[3057]	validation_0-mae:0.178389
[3058]	validation_0-mae:0.178387
[3059]	vali

[3278]	validation_0-mae:0.177738
[3279]	validation_0-mae:0.177735
[3280]	validation_0-mae:0.177728
[3281]	validation_0-mae:0.177724
[3282]	validation_0-mae:0.177722
[3283]	validation_0-mae:0.177721
[3284]	validation_0-mae:0.177719
[3285]	validation_0-mae:0.177714
[3286]	validation_0-mae:0.177713
[3287]	validation_0-mae:0.177711
[3288]	validation_0-mae:0.177708
[3289]	validation_0-mae:0.177707
[3290]	validation_0-mae:0.177703
[3291]	validation_0-mae:0.177698
[3292]	validation_0-mae:0.177692
[3293]	validation_0-mae:0.177688
[3294]	validation_0-mae:0.177686
[3295]	validation_0-mae:0.177685
[3296]	validation_0-mae:0.177685
[3297]	validation_0-mae:0.177676
[3298]	validation_0-mae:0.177671
[3299]	validation_0-mae:0.17767
[3300]	validation_0-mae:0.17767
[3301]	validation_0-mae:0.177666
[3302]	validation_0-mae:0.177663
[3303]	validation_0-mae:0.177659
[3304]	validation_0-mae:0.177656
[3305]	validation_0-mae:0.177651
[3306]	validation_0-mae:0.17765
[3307]	validation_0-mae:0.177648
[3308]	valida

[3528]	validation_0-mae:0.177075
[3529]	validation_0-mae:0.177072
[3530]	validation_0-mae:0.177071
[3531]	validation_0-mae:0.177066
[3532]	validation_0-mae:0.177064
[3533]	validation_0-mae:0.177062
[3534]	validation_0-mae:0.177057
[3535]	validation_0-mae:0.177055
[3536]	validation_0-mae:0.177053
[3537]	validation_0-mae:0.177052
[3538]	validation_0-mae:0.177051
[3539]	validation_0-mae:0.17705
[3540]	validation_0-mae:0.177044
[3541]	validation_0-mae:0.177044
[3542]	validation_0-mae:0.177044
[3543]	validation_0-mae:0.177041
[3544]	validation_0-mae:0.177039
[3545]	validation_0-mae:0.177035
[3546]	validation_0-mae:0.177032
[3547]	validation_0-mae:0.177029
[3548]	validation_0-mae:0.177028
[3549]	validation_0-mae:0.177022
[3550]	validation_0-mae:0.177019
[3551]	validation_0-mae:0.177017
[3552]	validation_0-mae:0.177014
[3553]	validation_0-mae:0.177011
[3554]	validation_0-mae:0.177011
[3555]	validation_0-mae:0.177007
[3556]	validation_0-mae:0.177006
[3557]	validation_0-mae:0.177004
[3558]	vali

[3778]	validation_0-mae:0.176504
[3779]	validation_0-mae:0.176502
[3780]	validation_0-mae:0.176496
[3781]	validation_0-mae:0.176493
[3782]	validation_0-mae:0.17649
[3783]	validation_0-mae:0.176485
[3784]	validation_0-mae:0.176482
[3785]	validation_0-mae:0.176479
[3786]	validation_0-mae:0.176479
[3787]	validation_0-mae:0.176477
[3788]	validation_0-mae:0.176474
[3789]	validation_0-mae:0.176472
[3790]	validation_0-mae:0.176469
[3791]	validation_0-mae:0.176465
[3792]	validation_0-mae:0.176464
[3793]	validation_0-mae:0.17646
[3794]	validation_0-mae:0.176459
[3795]	validation_0-mae:0.176456
[3796]	validation_0-mae:0.176453
[3797]	validation_0-mae:0.17645
[3798]	validation_0-mae:0.176448
[3799]	validation_0-mae:0.176448
[3800]	validation_0-mae:0.176445
[3801]	validation_0-mae:0.176444
[3802]	validation_0-mae:0.176444
[3803]	validation_0-mae:0.176442
[3804]	validation_0-mae:0.176438
[3805]	validation_0-mae:0.176436
[3806]	validation_0-mae:0.176435
[3807]	validation_0-mae:0.176434
[3808]	valida

In [ ]:
df_submission = pd.read_csv('./submissions/submission_31-07-2019.csv')

In [ ]:
#'''
better_coupling_types = ["1JHN"] # , "2JHH"]

for coupling_type in better_coupling_types:
    print(coupling_type)
    model_name = './featurebook/xgb_30.07/featurebook_%s.hdf5' % coupling_type
    
    xgb_model = xgb.XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, importance_type='gain',
       learning_rate=0.1, max_delta_step=0, max_depth=9, # was 9
       min_child_weight=1, missing=None, n_estimators=10000, n_jobs=-1,
       nthread=None, objective='reg:squarederror', random_state=101, reg_alpha=2,
       reg_lambda=0.2, scale_pos_weight=1, seed=None, silent=False, 
       subsample=1)
    
    xgb_model.load_model(model_name)
    
    df_train, df_test = feature_master(coupling_type)
    train_X, train_y, validation_X, validation_y = split(df_train)
    
    # Memory optimization
    memory_optimization([df_train, validation_X, validation_y])
    
    df_submission_temp = make_prediction(xgb_model, model_name, coupling_type, df_submission, df_test)
    df_submission = df_submission_temp

df_submission.to_csv(f"./submissions/submission_31-07-2019.csv", index=False)
#'''